In [1]:
import pandas as pd
import numpy as np

In [2]:
teacher_df = pd.read_csv('data/CY08MSP_TCH_KOR_VARS select.csv')

In [3]:
teacher_df = teacher_df.iloc[:, :-1]  # 마지막 열 제거
teacher_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3614 entries, 0 to 3613
Columns: 526 entries, CNT to VER_DAT
dtypes: float64(117), int64(407), object(2)
memory usage: 14.5+ MB


In [4]:
teacher_vars = pd.read_csv('data/TCH_VARS_original.csv')

In [5]:
teacher_vars.head()

,변수,설명,측도,결측값
0,CNT,Country code 3-character,명목(N),없음
1,CNTTCHID,Intl. Teacher ID,명목(N),없음
2,TCHTYPE,Teacher Type,명목(N),"7, 8, 9"
3,TC001Q01NA,Are you female or male?,명목(N),95 - 99
4,TC002Q01NA,How old are you?,척도(S),95 - 99


In [6]:
teacher_columns_selected = teacher_vars_selected['변수'].to_list()
teacher_columns_selected[:5]

np.True_

In [7]:
teacher_df_selected = teacher_df[teacher_columns_selected]
teacher_df_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   변수      117 non-null    object
dtypes: object(1)
memory usage: 1.0+ KB


In [8]:
teacher_df_selected.head()

['TCHTYPE', 'TC001Q01NA', 'TC002Q01NA', 'TC007Q02NA', 'TC215Q01JA']

In [9]:
9teacher_df_selected.describe(percentiles=[0.9, 0.92, 0.95, 0.99])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3614 entries, 0 to 3613
Columns: 117 entries, TCHTYPE to NEGSYMPT
dtypes: float64(33), int64(84)
memory usage: 3.2 MB


In [12]:
teacher_vars_selected_dtypes = teacher_vars_selected.merge(teacher_vars, on='변수', how='left')
teacher_vars_selected_dtypes

,변수
0,TCHTYPE
1,TC001Q01NA
2,TC002Q01NA
3,TC007Q02NA
4,TC215Q01JA
...,...
112,AUTONOMY
113,LEADSHIP
114,OCSTRESS
115,STRESS


**종속변수별 결측치값 확인**

In [13]:
delimiter = r' - |, '
re.split(delimiter, '7, 8, 9')

,변수,설명,측도,결측값
0,TCHTYPE,Teacher Type,명목(N),"7, 8, 9"
1,TC001Q01NA,Are you female or male?,명목(N),95 - 99
2,TC002Q01NA,How old are you?,척도(S),95 - 99
3,TC007Q02NA,How many years of work experience do you have:...,척도(S),95 - 99
4,TC215Q01JA,"During most recent complete calendar week, app...",명목(N),99995 - 99999
...,...,...,...,...
112,AUTONOMY,Teachers' work autonomy (WLE),척도(S),95.0000 - 99.0000
113,LEADSHIP,School leadership (WLE),척도(S),95.0000 - 99.0000
114,OCSTRESS,Occupational stress (WLE),척도(S),95.0000 - 99.0000
115,STRESS,Sources of stress (WLE),척도(S),95.0000 - 99.0000


종속변수별 결측치값을 실제로 결측치로 처리되도록 모두 `np.nan`으로 대체한다.

In [ ]:

delimiter = r' - |, ' # 구분자: ' - ' 또는 ', '

def parse_missing_codes(code_str):
    """'99, 999' -> [99.0, 999.0] 변환"""
    if pd.isna(code_str) or code_str == '':
        return np.array([np.nan])
    elif code_str == '없음':
        return np.array([np.nan])
    else:
        missing_range = [float(x.strip()) for x in re.split(delimiter, str(code_str))]
        return np.arange(missing_range[0], missing_range[-1] + 1)

# 변수명-결측값 어레이 사전 생성
missing_map = dict(zip(
    teacher_vars_selected_dtypes['변수'],
    teacher_vars_selected_dtypes['결측값'].apply(parse_missing_codes)
))

missing_map는 열별 결측치를 어레이로 지정한다.

In [58]:
missing_map

{'TCHTYPE': array([7., 8., 9.]),
 'TC001Q01NA': array([95., 96., 97., 98., 99.]),
 'TC002Q01NA': array([95., 96., 97., 98., 99.]),
 'TC007Q02NA': array([95., 96., 97., 98., 99.]),
 'TC215Q01JA': array([99995., 99996., 99997., 99998., 99999.]),
 'TC216Q01JA': array([995., 996., 997., 998., 999.]),
 'TC216Q02JA': array([995., 996., 997., 998., 999.]),
 'TC216Q03JA': array([995., 996., 997., 998., 999.]),
 'TC216Q04JA': array([995., 996., 997., 998., 999.]),
 'TC216Q05JA': array([995., 996., 997., 998., 999.]),
 'TC216Q06JA': array([995., 996., 997., 998., 999.]),
 'TC216Q07JA': array([995., 996., 997., 998., 999.]),
 'TC216Q08JA': array([995., 996., 997., 998., 999.]),
 'TC218Q01JA': array([95., 96., 97., 98., 99.]),
 'TC150Q01HA': array([95., 96., 97., 98., 99.]),
 'TC150Q02HA': array([95., 96., 97., 98., 99.]),
 'TC150Q03HA': array([95., 96., 97., 98., 99.]),
 'TC150Q04HA': array([95., 96., 97., 98., 99.]),
 'TC150Q06HA': array([95., 96., 97., 98., 99.]),
 'TC150Q08HA': array([95., 96.

열별 결측값을 일괄적으로 NaN 치환

In [ ]:
for col, codes in missing_map.items():
    if len(codes) > 1:
        teacher_df_selected.loc[:, col] = teacher_df_selected[col].replace(codes.tolist(), np.nan)

In [41]:
teacher_df_selected.describe()

,TCHTYPE,TC001Q01NA,TC002Q01NA,TC007Q02NA,TC215Q01JA,TC216Q01JA,TC216Q02JA,TC216Q03JA,TC216Q04JA,TC216Q05JA,...,CREATPED,CAPCON,AFFECT,TRUST,OVERLOAD,AUTONOMY,LEADSHIP,OCSTRESS,STRESS,NEGSYMPT
count,3614.000000,3609.000000,3608.000000,3604.000000,3603.000000,3568.000000,3545.000000,3449.000000,3413.000000,3439.000000,...,3568.000000,3567.000000,3570.000000,3570.000000,3566.000000,3573.000000,3559.000000,3568.000000,3561.000000,3559.000000
mean,1.684283,1.438903,43.868348,16.030244,46.504302,16.260650,8.667983,3.428530,3.487841,4.024426,...,-0.440153,-0.324506,-0.429214,-0.362764,0.140129,0.032868,-0.151507,0.233471,0.006706,0.170020
std,0.464865,0.496322,9.569938,10.129877,123.273670,6.119032,6.711867,3.674812,3.799902,4.835814,...,0.909236,0.933495,0.938867,0.894372,0.757571,1.053157,0.934281,0.695030,0.897948,0.934558
min,1.000000,1.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-3.804800,-3.894700,-3.139800,-3.009000,-3.506700,-3.401700,-2.030100,-3.761100,-2.721200,-2.325700
25%,1.000000,1.000000,36.000000,8.000000,25.000000,15.000000,5.000000,1.000000,2.000000,1.000000,...,-1.003425,-0.924100,-1.005800,-0.887800,-0.351600,-0.627800,-0.634800,-0.014900,-0.332400,-0.230350
50%,2.000000,1.000000,44.000000,15.500000,40.000000,16.000000,8.000000,2.000000,3.000000,3.000000,...,-0.636300,-0.137700,-0.054400,-0.475100,0.135000,-0.184300,-0.303400,0.209400,0.147300,0.243200
75%,2.000000,2.000000,51.000000,23.000000,50.000000,18.000000,10.000000,5.000000,5.000000,5.000000,...,0.018900,-0.137700,-0.054400,0.200700,0.581200,0.613100,0.400300,0.591500,0.503200,0.703000
max,2.000000,2.000000,64.000000,46.000000,3310.000000,100.000000,80.000000,70.000000,60.000000,100.000000,...,1.766800,2.330100,1.740600,1.557900,3.371300,2.565200,2.754700,1.465000,3.682900,3.838000


열별 결측치 개수를 확인한다.
종속 변수로 사용될 FEEDBINSTR와 FEEDBINSTR에 각각 583개의 결측치가 존재한다.

In [56]:
teacher_df_selected.isnull().sum().sort_values(ascending=False)

TC150Q06HA    3134
TC150Q04HA    3127
TC150Q03HA    3122
TC150Q02HA    3121
TC204Q03HA    3119
              ... 
STTMG9           0
STTMG11          0
TC045Q01         0
TC045Q02         0
TC045Q03         0
Length: 117, dtype: int64

In [66]:
teacher_df1 = teacher_df_selected.dropna(subset=['ADAPTINSTR', 'FEEDBINSTR'])

In [68]:
teacher_df_selected.shape

(3614, 117)

In [67]:
teacher_df1.shape

(3030, 117)

In [57]:
teacher_vars_selected_dtypes['측도'].value_counts()

측도
순서(O)    44
명목(N)    38
척도(S)    35
Name: count, dtype: int64

In [ ]:
cols = teacher_df_selected.columns[(~mask_scale).to_numpy()]
teacher_df_selected.loc[:, cols] = teacher_df_selected.loc[:, cols].astype(str)

In [ ]:
teacher_df_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3614 entries, 0 to 3613
Columns: 117 entries, TCHTYPE to NEGSYMPT
dtypes: float64(32), int64(3), object(82)
memory usage: 3.2+ MB


In [ ]:
teacher_df_selected.select_dtypes(include='int64')

In [ ]:
teacher_df_selected.select_dtypes(include='float64')

In [ ]:
teacher_df_selected.describe()

In [ ]:
teacher_df_selected['TC258Q01JA'].value_counts()

In [ ]:
teacher_df_selected['PRPDT'].value_counts()

In [ ]:
teacher_df_selected.loc[:, 'ADAPTINSTR']

In [ ]:
targets = ['ADAPTINSTR', 'FEEDBINSTR']

In [ ]:
teacher_df_selected.loc[:, targets].describe()

In [ ]:
teacher_df_selected.loc[:, 'ADAPTINSTR'].value_counts()

In [ ]:
teacher_df_selected.loc[:, 'FEEDBINSTR'].value_counts()

FEEDBINSTR
-0.1391     784
 99.0000    583
 1.4961     337
-1.6892     307
-0.7286     116
           ... 
-0.4544       1
-1.6384       1
 0.0197       1
-1.0032       1
-2.5998       1
Name: count, Length: 249, dtype: int64

In [ ]:
# 모델
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# 전처리
from sklearn.preprocessing import OneHotEncoder

# 컬럼 변환: 파이프라인 활용
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer

from sklearn.pipeline import Pipeline

# 모델 평가
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay


In [ ]:
teacher_df_selected.isna().sum().sum()

## RandomForest 학습

In [ ]:
X_2022 = teacher_df_selected.drop(targets, axis=1).copy()
y_2022 = teacher_df_selected.loc[:, targets].copy()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_2022, y_2022, test_size=0.2, random_state=42)